https://github.com/kjahan/community/blob/master/README.md を参照

_GirvanNewmanGetModularityの部分に関してIMG_5358の画像を確認

In [ ]:
import networkx as nx
import math
import csv
import random as rand
import sys

_DEBUG_ = False

                                                                                                               
def buildG(G, list):                                                                                            
    for line in list:
       G.add_edge(line[0], line[1],weight=float(line[2]))                                                       #これでファイルから重み付き無向グラフの情報を打ち込む 元々int関数がついてたが削除
                                                                                                                #もしファイルの情報に重みが付与されていなかったら重みを自動で1とする。
                                                                                                                #この作業はGの中にある塊の数が一つ増えるまでエッジを減らし続ける。得られる成果はエッジが減ったG
                                                                                                                # This method keeps removing edges from Graph until one of the connected components of Graph splits into two
                                                                                                                # compute the edge betweenness
def CmtyGirvanNewmanStep(G): #③
    if _DEBUG_:
        print("Running CmtyGirvanNewmanStep method ...")
    init_ncomp = nx.number_connected_components(G)                                                              #number of components　連結グループの数を取得
    ncomp = init_ncomp                                                                                          #ncompを更新し続けてある塊を分割できるまで繰り返す。
    while ncomp <= init_ncomp:
        bw = nx.edge_betweenness_centrality(G, weight='weight')                                                 #edge betweenness for G
                                                                                                                #find the edge with max centrality その段階のグラフで最大の中心性を持つエッジを取得。
                                                                                                                #bwの形式は形式は{('エッジ', 'エッジ'): edge_betweenness_centralityの値の値, ...}
        max_ = max(bw.values())
                                                                                                                #find the edge with the highest centrality and remove all of them if there is more than one!
        for k, v in bw.items():                                                                                 #この場合kはインデックスでvが中身
            if float(v) == max_:
                G.remove_edge(k[0],k[1])                                                                        #remove the central edge
        ncomp = nx.number_connected_components(G)                                                               #recalculate the no of components

                                                                                                                #渡されたグラフGのモジュラリティーを計算して引数Modとして渡す
                                                                                                                # This method compute the modularity of current split
def _GirvanNewmanGetModularity(G, deg_, m_): #②
    New_A = nx.adjacency_matrix(G)                                                                              #データは['エッジ', 'エッジ', 重み]というリストが並んでいる。
    New_deg = {}
    New_deg = UpdateDeg(New_A, G.nodes())                                                                       #（重複）UpdateDegでは辞書型で{'ノード名': 重みの総和, ...}が取得できる。
                                                                                                                #Let's compute the Q
    comps = nx.connected_components(G)                                                                                                             #list of components データの形式の確認は困難 
    print('Number of communities in decomposed G: {}'.format(nx.number_connected_components(G)))                #元々Noと記載されてたが、Numberに修正
    Mod = 0                                                                                                     #Modularity of a given partitionning
    for c in comps:
        EWC = 0                                                                                                 #number of edges within a community
        RE = 0                                                                                                  #number of random edges
        for u in c:
            EWC += New_deg[u]     #EWCはその連結部のノードの最新の重みの総和
            RE += deg_[u]         #REはその連結部のノードの初期の重みの総和                                           #count the probability of a random edge
        Mod += ( float(EWC) - float(RE*RE)/float(2*m_) )                                                        #モジュラリティーの計算
    Mod = Mod/float(2*m_) 
    if _DEBUG_:
        print("Modularity: {}".format(Mod))
    return Mod

                                                                                                                #（重複）UpdateDegでは辞書型で{'ノード名': 重みの総和, ...}がdeg_dictに格納されている。
def UpdateDeg(A, nodes): #                                                                                      #nodes = G.nodes()でノードが並ぶリストが格納される。
    deg_dict = {}
    n = len(nodes)                                                                                              #len(A) ---> some ppl get issues when trying len() on sparse matrixes!
    B = A.sum(axis = 1)                                                                                         #グラフのノードが持つそれぞれのエッジの重みを足し合わせたものが格納され、それはノードごとにあり、ノードの番号順に並べられる。
    i = 0
    for node_id in list(nodes): 
        deg_dict[node_id] = B[i, 0]                                                                             #Bは縦に一列なのでB[i, 0]はi番目の要素のこと。つまりdeg_dictは
        i += 1
    return deg_dict
   


                                                                                                                # This method runs GirvanNewman algorithm and find the best community split by maximizing modularity measure
def runGirvanNewman(G, Orig_deg, m_): #④
                                                                                                                #let's find the best split of the graph
    BestQ = 0.0
    Q = 0.0
    while True:    
        CmtyGirvanNewmanStep(G)                                                                                 #エッジが減らされて分割されたGが得られる。
        Q = _GirvanNewmanGetModularity(G, Orig_deg, m_);                                                        #Gのモジュラリティーを取得
        print("Modularity of decomposed G: {}".format(Q))
        if Q > BestQ:
            BestQ = Q                                                                                           #最高値のモジュラリティー値を取得
            Bestcomps = list(nx.connected_components(G))    #Best Split
            print("Identified components: {}".format(Bestcomps))
        if G.number_of_edges() == 0:                                                                            #分割し終えたら処理を抜ける
            break
    if BestQ > 0.0:                                                                                             #分割作業が終わって最大のモジュラリティーを得たので表示
        print("Max modularity found (Q): {} and number of communities: {}".format(BestQ, len(Bestcomps)))
        print("Graph communities: {}".format(Bestcomps))
    else:
        print("Max modularity (Q):", BestQ)                                                                     #モジュラリティーが0の場合は別処理


def main(argv):
    if len(argv) < 2:
        sys.stderr.write("Usage: %s <input graph>\n" % (argv[0],))
        return 1
    graph_fn = argv[1]
    G = nx.Graph()                                                                                              #let's create the graph first グラフ情報を取り込んでグラフGを作成。
    buildG(G, graph_fn, ',')                                                                                    #グラフのデータは’,’で区切られている
    
    if _DEBUG_:
        print('G nodes: {} & G no of nodes: {}'.format(G.nodes(), G.number_of_nodes()))
    
    n = G.number_of_nodes()                                                                                     #|V|
    A = nx.adjacency_matrix(G)                                                                                  #adjacenct matrix

    m_ = 0.0                                                                                                    #the weighted version for number of edges
    for i in range(0,n):                                                                                        #エッジの数だけ処理
        for j in range(0,n):                                                                                    #エッジの数だけ処理
            m_ += A[i,j]                                                                                        #これでAの隣接行列のiからjに行くエッジの重みを取得、ちなAのノードは数字に変換される。
    m_ = m_/2.0                                                                                                 #一つのエッジについて2回計算しているから2で割っている。
                                                                                                                #ここではグラフの全ての重みを足し合わせた値：m_を得た。
    if _DEBUG_:
        print("m: {}".format(m_))

                                                                                                                #calculate the weighted degree for each node
    Orig_deg = {}
    Orig_deg = UpdateDeg(A, G.nodes())                                                                          #G.nodes()でノードが並ぶリストが作られる。
                                                                                                                #(重複)上のOrig_degには辞書型で{'ノード名': 重みの総和, ...}が格納されている。
    #run Newman alg
    runGirvanNewman(G, Orig_deg, m_)

In [ ]:
G = nx.Graph()  #let's create the graph first
buildG(G, matrix_con) #ここに三次元リストを入れて実行！
    
if _DEBUG_:
  print('G nodes: {} & G no of nodes: {}'.format(G.nodes(), G.number_of_nodes()))
    
n = G.number_of_nodes()    #|V|
A = nx.adjacency_matrix(G)    #adjacenct matrix

m_ = 0.0    #the weighted version for number of edges
for i in range(0,n):
  for j in range(0,n):
    m_ += A[i,j]
m_ = m_/2.0
if _DEBUG_:
        print("m: {}".format(m_))

    #calculate the weighted degree for each node
Orig_deg = {}
Orig_deg = UpdateDeg(A, G.nodes())

    #run Newman alg
runGirvanNewman(G, Orig_deg, m_)

In [ ]:
#community detection final

import networkx as nx
import math
import csv
import random as rand
import sys

_DEBUG_ = False

                                                                                                               
def buildG(G, list):                                                                                            
    for line in list:
       G.add_edge(line[0], line[1],weight=float(line[2]))                                                       #これでファイルから重み付き無向グラフの情報を打ち込む 元々int関数がついてたが削除
                                                                                                                #もしファイルの情報に重みが付与されていなかったら重みを自動で1とする。
                                                                                                                #この作業はGの中にある塊の数が一つ増えるまでエッジを減らし続ける。得られる成果はエッジが減ったG
                                                                                                                # This method keeps removing edges from Graph until one of the connected components of Graph splits into two
                                                                                                                # compute the edge betweenness
def CmtyGirvanNewmanStep(G): #③
    if _DEBUG_:
        print("Running CmtyGirvanNewmanStep method ...")
    init_ncomp = nx.number_connected_components(G)     
    ncomp = init_ncomp                                      
    while ncomp <= init_ncomp:
        bw = nx.edge_betweenness_centrality(G, weight='weight')                              
        max_ = max(bw.values())                                    
        for k, v in bw.items():                    
            if float(v) == max_:
                G.remove_edge(k[0],k[1])                    
        ncomp = nx.number_connected_components(G)                                                               #recalculate the no of components

                                                                                                                #渡されたグラフGのモジュラリティーを計算して引数Modとして渡す
                                                                                                                # This method compute the modularity of current split
def _GirvanNewmanGetModularity(G, deg_, m_): #②
    New_A = nx.adjacency_matrix(G)                                                                              #データは['エッジ', 'エッジ', 重み]というリストが並んでいる。
    New_deg = {}
    New_deg = UpdateDeg(New_A, G.nodes())                                                                       #（重複）UpdateDegでは辞書型で{'ノード名': 重みの総和, ...}が取得できる。
                                                                                                                #Let's compute the Q
    comps = nx.connected_components(G)                                                                                                             #list of components データの形式の確認は困難 
    # print('Number of communities in decomposed G: {}'.format(nx.number_connected_components(G)))                #元々Noと記載されてたが、Numberに修正
    Mod = 0                                                                                                     #Modularity of a given partitionning
    for c in comps:
        EWC = 0                                                                                                 #number of edges within a community
        RE = 0                                                                                                  #number of random edges
        for u in c:
            EWC += New_deg[u]     #EWCはその連結部のノードの最新の重みの総和
            RE += deg_[u]         #REはその連結部のノードの初期の重みの総和                                           #count the probability of a random edge
        Mod += ( float(EWC) - float(RE*RE)/float(2*m_) )                                                        #モジュラリティーの計算
    Mod = Mod/float(2*m_) 
    if _DEBUG_:
        print("Modularity: {}".format(Mod))
    return Mod

                                                                                                                #（重複）UpdateDegでは辞書型で{'ノード名': 重みの総和, ...}がdeg_dictに格納されている。
def UpdateDeg(A, nodes): #                                                                                      #nodes = G.nodes()でノードが並ぶリストが格納される。
    deg_dict = {}
    n = len(nodes)                                                                                              #len(A) ---> some ppl get issues when trying len() on sparse matrixes!
    B = A.sum(axis = 1)                                                                                         #グラフのノードが持つそれぞれのエッジの重みを足し合わせたものが格納され、それはノードごとにあり、ノードの番号順に並べられる。
    i = 0
    for node_id in list(nodes): 
        deg_dict[node_id] = B[i, 0]                                                                             #Bは縦に一列なのでB[i, 0]はi番目の要素のこと。つまりdeg_dictは
        i += 1
    return deg_dict
   


                                                                                                                # This method runs GirvanNewman algorithm and find the best community split by maximizing modularity measure
def runGirvanNewman(G, Orig_deg, m_): #④
                                                                                                                #let's find the best split of the graph
    BestQ = 0.0
    Q = 0.0
    while True:    
        CmtyGirvanNewmanStep(G)                                                                                 #エッジが減らされて分割されたGが得られる。
        Q = _GirvanNewmanGetModularity(G, Orig_deg, m_);                                                        #Gのモジュラリティーを取得
        # print("Modularity of decomposed G: {}".format(Q))
        if Q > BestQ:
            BestQ = Q                                                                                           #最高値のモジュラリティー値を取得
            Bestcomps = list(nx.connected_components(G))    #Best Split
            # print("Identified components: {}".format(Bestcomps))
        if G.number_of_edges() == 0:                                                                            #分割し終えたら処理を抜ける
            break
    if BestQ > 0.0:                                                                                             #分割作業が終わって最大のモジュラリティーを得たので表示
        print("Max modularity found (Q): {} and number of communities: {}".format(BestQ, len(Bestcomps)))
        print("Graph communities: {}".format(Bestcomps))
    else:
        print("Max modularity (Q):", BestQ)                                                                     #モジュラリティーが0の場合は別処理


def build(graph_fn):
    G = nx.Graph()                                                                                              #let's create the graph first グラフ情報を取り込んでグラフGを作成。
    buildG(G, graph_fn)                                                                                    #グラフのデータは’,’で区切られている
    
    if _DEBUG_:
        print('G nodes: {} & G no of nodes: {}'.format(G.nodes(), G.number_of_nodes()))
    
    n = G.number_of_nodes()                                                                                     #|V|
    A = nx.adjacency_matrix(G)                                                                                  #adjacenct matrix

    m_ = 0.0                                                                                                    #the weighted version for number of edges
    for i in range(0,n):                                                                                        #エッジの数だけ処理
        for j in range(0,n):                                                                                    #エッジの数だけ処理
            m_ += A[i,j]                                                                                        #これでAの隣接行列のiからjに行くエッジの重みを取得、ちなAのノードは数字に変換される。
    m_ = m_/2.0                                                                                                 #一つのエッジについて2回計算しているから2で割っている。
                                                                                                                #ここではグラフの全ての重みを足し合わせた値：m_を得た。
    if _DEBUG_:
        print("m: {}".format(m_))

                                                                                                                #calculate the weighted degree for each node
    Orig_deg = {}
    Orig_deg = UpdateDeg(A, G.nodes())                                                                          #G.nodes()でノードが並ぶリストが作られる。
                                                                                                                #(重複)上のOrig_degには辞書型で{'ノード名': 重みの総和, ...}が格納されている。
    #run Newman alg
    runGirvanNewman(G, Orig_deg, m_)

In [ ]:
build(matrix_condt)

In [ ]:
sum_engname_list1 = ['Gendai_biz', 'Okinawa', 'Mainichi', 'Asahi', 'Jiji', 'Nikkei', 'Sankei', 'Tokyo', 'Yomiuri', 'Zakzak', 'Nhk', 'Akahata', 'Nikkan', 'Agora', 'Anonymous', 'Hamusoku', 'Seijichishin', 'Litera', 'Share_News_Japan', 'Moeruasia', 'BuzzFeed_Japan', 'Yahoo_News']
sum_engname_list2 = ['Jiyuminsyu', 'Rikkenminsyu', 'Komei', 'Kyosan', 'Ishin', 'Kokuminminsyu', 'Syakaiminsyu', 'Reiwa']
sum_engname_list3 = ['Jiyuminsyu_Leader', 'Rikkenminsyu_Leader', 'Komei_Leader', 'Kyosan_Leader', 'Ishin_Leader', 'Kokuminminsyu_Leader', 'Syakaiminsyu_Leader', 'Reiwa_Leader', 'Jiyuminsyu_Suga', 'Jiyuminsyu_Kono', 'Rikkenminsyu_Edano', 'Rikkenminsyu_Renho', 'Ishin_Yoshimura', 'Ishin_Hashimoto']

#セクション4.1.1
#グラフリスト 
Jaccard係数 = 0.05

matrix = matrix_j

matrix_conj = []
for i in range(len(matrix)):
  if matrix[i][2] >= Jaccard係数 and matrix[i][2] != 1:
    matrix_conj.append(matrix[i])

H = nx.Graph()
for n in sum_engname_list:
    H.add_node(n)
size =[((len(i)-1)/500) for i in sum_list]
H.add_weighted_edges_from(matrix_conj)
edge_width = [d["weight"] * 10  for (u, v, d) in H.edges(data=True)]

c = []
for i in H.nodes():
  if i in sum_engname_list1:
    c.append('red')
  elif i in sum_engname_list2:
    c.append('blue')
  elif i in sum_engname_list3:
    c.append('green')
  elif i in sum_engname_list4:
    c.append('magenta')
  else:
    c.append('yellow')

# pos = nx.spring_layout(H, k=0.4) 
pos=nx.nx_pydot.graphviz_layout(H)
plt.figure(figsize=(18,12))
nx.draw_networkx(H, pos,node_size = size, node_color=c, arrows=False, width= edge_width, connectionstyle="arc3, rad=0.1")

In [ ]:
#セクション4.1.1 
#グラフリスト 
Jaccard係数 = 0.06

import networkx as nx
import matplotlib.pyplot as plt

matrix_conj = []
for i in range(len(matrix_j)):
  if matrix_j[i][2] >= Jaccard係数 and matrix_j[i][2] != 1: #右の条件式でアカウントの自己ループをなくす
    matrix_conj.append(matrix_j[i])

H = nx.Graph()
H.add_weighted_edges_from(matrix_conj)
nx.draw_kamada_kawai(H, with_labels=True)